In [3]:
%matplotlib inline

import gc

import numpy as np
import matplotlib.pyplot as plt

from scipy.signal import convolve2d

from skimage import data, io, color
from skimage.transform import resize

plt.rcParams['figure.figsize'] = [8, 8]

def show_plot(image):
    plt.imshow(image, cmap=plt.get_cmap("gray"))
    plt.colorbar()
    plt.show()

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


##### Decimation matrix - assumes only taking 4 neighbours and that $L = M*2$

In [ ]:
# l = 8
# m = 4
# d = l // m

# d_matrix = np.zeros((m**2, l**2))

# tile = np.repeat((1, 0, 1), (2, l - 2, 2)) # assuming taking 2 neighbours per dimension
# t_len = tile.shape[0]

# for p in np.arange(l//4):
#     p_offset = p * l # 8
#     for q in np.arange(m):
#         d_matrix[q+ p_offset//2, q*d + p_offset*2 : q*d+t_len + p_offset*2] = tile # top-left quadrant
#         r_offset = m**2 // 2
#         c_offset = l**2 // 2
#         d_matrix[q+r_offset + p_offset//2, q*d+c_offset + p_offset*2: q*d+t_len+c_offset + p_offset*2] = tile # bottom-right quadrant

In [ ]:
# x = np.arange(1, 65) # 1..64
# print(x.reshape(8, 8))

# y = np.matmul(d_matrix * 1/4, x)
# print(y.reshape(4, 4))

##### Sharpening matrix - Laplacian operator

In [ ]:
# x_true = color.rgb2gray(data.astronaut())
# x = resize(x_true, (100, 100), anti_aliasing=True)
# show_plot(x)

# laplace_deblur = np.array([[0, -1,  0], [-1,  4, -1], [0, -1,  0]], dtype=np.float32)
# show_plot(convolve2d(x, laplace_deblur, mode="same") )

# NOTE: Assumes you will be matrix multiplying this against an image of l*l, padded by one border of zeros
# Adam: see if this is correct, compare this against the convolve2d results from above

l = 200
laplacian = np.array([[1.0, 2.0, 1.0], [2.0, 4.0, 2.0], [1.0, 2.0, 1.0]], dtype=np.float32)
laplacian_dim = laplacian.shape[0]
padding = laplacian_dim - 1 # s_matrix padding for out of bounds convolution
laplacian = laplacian.flatten()
strided_laplacian = np.insert(laplacian, laplacian_dim, np.repeat(0, l-1))
strided_laplacian = np.insert(strided_laplacian, laplacian_dim*2+l-1, np.repeat(0, l-1))
print("%f kilobytes approx " % (strided_laplacian.nbytes/(2**10)))
s_matrix = np.zeros(((l+padding)**2, (l+padding)**2), dtype=np.float32)
print("%f gigabytes approx" % (s_matrix.nbytes/(2**30)))
# sr_offset = l+padding+padding//2

# for sr in np.arange(l):
#     for r in np.arange(l):
#         row_offset = (sr*l)+(l+padding+padding//2)+(padding*sr)
#         col_offset = sr*(l+padding)+r
#         s_matrix[row_offset+r, col_offset:col_offset+strided_laplacian.shape[0]] = strided_laplacian
        
# show_plot(s_matrix)